# 1 : Librairies Python

In [1]:
# Librairies et options :
# Divers :
import pandas as pd
import random
import datetime
import requests
import json
import numpy as np
import tqdm
import time
import pytz

# Importer les codes depuis un autre notebook :
%run Codes.ipynb

# Importer les codes depuis un fichier .env :
from dotenv import load_dotenv
import os
load_dotenv()

# Création et lecture des BDD postgresl :
from sqlalchemy import create_engine
import psycopg2
from sqlalchemy.types import Integer, BigInteger, JSON

# 2 : Clés API et BDD :

In [2]:
# Informations API : https://weatherlink.github.io/v2-api/
# Clés API :
APIKey = os.getenv("APIKey")
APISecret = os.getenv("APISecret")
stationID = os.getenv("stationID")

In [3]:
# Paramètres de connexion à la base de données PostgreSQL en local :
host = os.getenv("host")
database = os.getenv("database")
user = os.getenv("user")
password = os.getenv("password")
nomTable = os.getenv("nomTable")

# 3 : Récupération de la TS la plus récente

In [4]:
# Connexion à la base de données
conn = psycopg2.connect(dbname = database, user = user, password = password , host = host)

# Création d'un curseur : permet d'exécuter des commandes SQL sur la base de données.
cur = conn.cursor()

# Exécution d'une requête SQL pour sélectionner les données de ma_table
cur.execute(f"SELECT ts FROM {nomTable} ORDER BY ts DESC LIMIT 1")

# Récupération des données dans une liste de tuples
data = cur.fetchall()

# Création d'un DataFrame à partir des données
LastLoadTimestamp = pd.DataFrame(data, columns=[desc[0] for desc in cur.description]).values[0][0]

# Fermeture du curseur et de la connexion
cur.close()
conn.close()

LastLoadTimestamp

1713218400

# 4 : Ouverture de l'API

In [ ]:
# Date d'aujourd'hui à minuit :
Today = datetime.date.today()
TodayMidnight = datetime.datetime.combine(Today, datetime.time.min)

# Dernier enregistrement dans la BDD :
LastLoadTimestamp = df.values[0][0]

# Convertir la date :
TodayMidnightTimestamp = int(TodayMidnight.timestamp())

print("Le timestamp correspondant à la date du {} est :".format(TodayMidnight), TodayMidnightTimestamp)

In [ ]:
# DataFrame historiques :
dfNew = pd.DataFrame()

# Nb de jours à récupérer :
NbJours = int((TodayMidnightTimestamp - LastLoadTimestamp) / 86400)

for i in tqdm.tqdm(range(NbJours)):
    StartTime = LastLoadTimestamp + i * 86400
    EndTime = StartTime + 86400
    
    # Lien de la request : 
    link = 'https://api.weatherlink.com/v2/historic/{}?api-key={}&start-timestamp={}&end-timestamp={}'.format(StationID, APIKey, StartTime, EndTime)
    headers = {'X-Api-Secret' : APISecret}

    # Requête :
    r = requests.get(link, headers=headers)

    # Si la requête a réussi :
    if r.status_code == 200:
        # Lecture de la request en json :
        data = r.json()

        # Transformation en DF : 
        dfJour = pd.DataFrame(data)
        dfJour = dfJour[['station_id','sensors']]

        # Récupération des valeurs se trouvant dans sensors :
        dfSensors = pd.json_normalize(data['sensors'][0]['data'])
    
        # Récupération des json sur une colonne :
        dfJour = pd.DataFrame({'station_id': data['station_id'], 'infos_json' : data['sensors'][0]['data']})
    
        # Convertir les objets JSON en chaînes de caractères JSON :
        dfJour['infos_json'] = dfJour['infos_json'].apply(json.dumps)

        # Concat des données :
        dfJour = pd.concat([dfJour, dfSensors], axis=1)
        
        # Concaténation des données :
        dfNew = pd.concat([dfNew, dfJour], ignore_index=True)
    else:
        print("La requête {} a échoué avec le code d'erreur {}".format(link, r.status_code))

In [ ]:
# Data Frame Finale :
dfNew.head()

# 3 : Transfert sur PostgreSQL

In [ ]:
# Début du chronomètre :
start_time = time.time()

# Création de la chaîne de connexion PostgreSQL :
conn_str = f"postgresql://{user}:{password}@{host}/{database}"

# Création de la connexion à la base de données PostgreSQL :
engine = create_engine(conn_str)

# Définir les types de données pour chaque colonne :
dtype = {'station_id': Integer(),
         'ts': BigInteger(),
         'infos_json': JSON}

# Ajouter le DataFrame dans la base de données PostgreSQL :
dfNew.to_sql(nomTable, engine, if_exists = 'append', index=False, dtype=dtype)

# Fermeture de la connexion :
engine.dispose()

# Fin du chronomètre :
end_time = time.time()

# Calcul de la durée d'exécution :
execution_time = end_time - start_time

# Affichage de la durée d'exécution :
print("Durée d'exécution :", round(execution_time/60,2), "minutes")